In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as smp
from scipy.integrate import odeint
import plotly.graph_objects as go
from IPython.display import HTML
from sympy.matrices import Matrix
from sympy.vector import Vector
import plotly.express as px
from ipywidgets import interact
from ipywidgets import interactive
from ipywidgets import widgets
from scipy.integrate import odeint
from IPython.display import HTML
from scipy.integrate import solve_ivp
import plotly.io as pio

Easy axis in X-direction.  Hard axis in Z direction.
There is no magnetic field. Spin accumulation in +z direction.

We turn of the spin accumulation at a specific point!

#### Free Energy Density
We start by the free energy density in terms of unit magnetizations $\textbf{m}$:

$$ \frac{\epsilon}{\hbar\gamma_{0}} =-\textbf{m}\cdot \textbf{H}-K m_{x}^2 +D m_{z}^2 $$
#### LLG Equation
$$\frac{d\vec{\textbf{m}}}{dt}= \gamma_{0} (\vec{\textbf{m}} \times H^{eff})+ \alpha_{0} \vec{\textbf{m}} \times\frac{d\vec{\textbf{m}}}{dt} +\gamma_{0}H_{DL}\vec{\textbf{m}}\times[ \vec{\textbf{m}}\times \hat{\sigma}] $$

We define the necessary variables for symbolic calculation using Sympy:

In [2]:
t = smp.symbols('t', real=True)
Hx, Hy, Hz, a, g, FL, DL, Hex, Kx,Dz= smp.symbols('Hx Hy Hz a g H_{FL} H_{DL} H_{ex} K_{x} D_{z} ', real=True)
mx, my, mz = smp.symbols(r'm_{x} m_{y} m_{z}', cls=smp.Function)
mx = mx(t)
my = my(t)
mz = mz(t)
# Derivatives
mx_d = smp.diff(mx,t)
my_d = smp.diff(my,t)
mz_d = smp.diff(mz,t)

m=Matrix([mx,my,mz])

m_d=Matrix([mx_d,my_d,mz_d])
H=Matrix([Hx,Hy,Hz])

X=Matrix([1,0,0])
Y=Matrix([0,1,0])
Z=Matrix([0,0,1])

Then we set the values of material parameters such as exchange, Gilbert damping, DMI and anisotropy:

In [3]:
a_value=0.06

Kx_value=0.01
Dz_value=0.1
g_value=1

DL_value=0.01

Hx_value,Hy_value,Hz_value=0,0,0

e=Matrix([1,0,0])

Setting the free energy and calculating effective fields:

In [4]:
FE= -1*Kx*((X.dot(m))**2)+Dz*((Z.dot(m))**2) -H.dot(m)
Heff=smp.diff(FE,m)


LLG Equation:

$$\frac{d\vec{m}}{dt}= \gamma_{0} (\vec{\textbf{m}} \times H^{eff})+ \alpha_{0} \vec{\textbf{m}} \times\frac{d\vec{\textbf{m}}}{dt} +\gamma_{0}H_{DL}\vec{\textbf{m}}\times[ \vec{\textbf{m}}\times \hat{\sigma}] $$

In [5]:
llg=-m_d + g*(m.cross(Heff))+a*(m.cross(m_d))+g*DL*m.cross(m.cross(Z))

We solve symbollically both equations for $\frac{d\vec{m}_{1,2}}{dt}$: (we don't solve the differential equation yet. We just have a symbollic expression for $\frac{d\vec{m}_{1,2}}{dt}$)

In [6]:
sols= smp.solve(llg, (mx_d, my_d, mz_d),simplify=True, rational=False)

We evaluate the symbolic expressions:

In [7]:
mx_df = smp.lambdify((t,m), sols[mx_d].subs([(g,g_value),(a,a_value),(Kx,Kx_value),(Dz,Dz_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))
my_df = smp.lambdify((t,m), sols[my_d].subs([(g,g_value),(a,a_value),(Kx,Kx_value),(Dz,Dz_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))
mz_df = smp.lambdify((t,m), sols[mz_d].subs([(g,g_value),(a,a_value),(Kx,Kx_value),(Dz,Dz_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))

We define a function and inital conditions so that we can solve the ODES numerically:

In [8]:
def dSdt(t,S):
    mx,my,mz  = S
    return [
        mx_df(t,[mx,my,mz]),
        my_df(t,[mx,my,mz]),
        mz_df(t,[mx,my,mz]),
    ]

theta=90*(np.pi/180)
phi=0*(np.pi/180)

x=np.sin(theta)*np.cos(phi)
y=np.sin(theta)*np.sin(phi)
z=np.cos(theta)



init_cond=(x,y,z)

We solve the coupled ODEs using ODEINT function:

In [9]:

t = np.linspace(0, 1000, 5000)

sol = odeint(dSdt, y0=init_cond, t=t, tfirst=True)
sol=sol.T

We plot the graphics:

In [10]:
X1s=sol[0]
Y1s=sol[1]
Z1s=sol[2]

fig1 = go.FigureWidget()
#NEEL VECTOR1
x1 = np.array([0,X1s[0]])
y1 = np.array([0,Y1s[0]])
z1 = np.array([0,Z1s[0]])
f=0.95
fig1.add_scatter3d(x=x1*f, y=y1*f, z=z1*f,opacity=0.75,
    marker=dict(size=0.1,color='black'),
    line=dict(color='black', width=4 ),
    name="$m$",showlegend=False)
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
    sizeref=0.1,colorscale=[[0, 'black'], [1,'black']])


Y=[0,1,0]
m=[X1s[0],Y1s[0],Z1s[0]]
DL=np.cross(m,np.cross(m,Y))
DL=DL/(5*np.linalg.norm(DL))
x = np.array([X1s[0],X1s[0]+DL[0]])
y = np.array([Y1s[0],Y1s[0]+DL[1]])
z = np.array([Z1s[0],Z1s[0]+DL[2]])

#Damping-like Torque
#fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.9,
#    marker=dict(size=0.1,color='black'),
#    line=dict(color='brown', width=5 ),
#    name="$ \\tau_{DL}$",showlegend=True)

fig1.update_layout(
    height=700,
    width=900,

    autosize=True,
    scene=dict(camera=dict(up=dict(x=0,y=0, z=1 ), eye=dict(x=1,y=-1,z=1,)),
               xaxis_title='mx',
               yaxis_title='my',
               zaxis_title='mz',
               xaxis = dict(nticks=3, range=[-1,1]),
               yaxis = dict(nticks=3, range=[-1,1]),
               zaxis = dict(nticks=3, range=[-1,1]),
        aspectratio = dict( x=1, y=1, z=1),
        aspectmode = 'manual'))
#CIRCLE IN XY PLANE
C1=1
phi=np.linspace(0,2*np.pi,100)
x=C1*np.cos(phi)
y=C1*np.sin(phi)
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN XZ PLANE
phi=np.linspace(0,2*np.pi,100)
x=C1*np.cos(phi)
y=C1*np.zeros(len(x))
z=C1*np.sin(phi)
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

#CIRCLE IN YZ PLANE
phi=np.linspace(0,2*np.pi,100)
x=C1*np.zeros(len(x))
y=C1*np.cos(phi)
z=C1*np.sin(phi)
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=C1*np.zeros(len(x))
y=C1*np.zeros(len(x))
z=phi
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=C1*np.zeros(len(x))
y=phi
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=phi
y=C1*np.zeros(len(x))
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

# UNIT SPHERE
theta, phi = np.mgrid[0:2*np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.sin(phi)
y = np.sin(theta)*np.cos(phi)
z = np.cos(theta)

fig1.add_surface(x=x,y=y,z=z,opacity=0.07,colorscale=[[0, 'purple'], [1,'orange']],showlegend=False, showscale = False)

fig1.add_scatter3d(x=X1s, y=Y1s, z=Z1s,opacity=0.25,
    marker=dict(size=0.1,color='red'),
    line=dict(color='red', width=3),
    showlegend=False)

#External Field1
#norm=2*np.sqrt(float(Hx_value**2+Hy_value**2+Hz_value**2))
#x1 = np.array([0,Hx_value/norm])
#y1 = np.array([0,Hy_value/norm])
#z1 = np.array([0,Hz_value/norm])

#fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
#    marker=dict(size=0.1,color='black'),
#    line=dict(color='green', width=6 ),
#    name="$H_{applied}$")
#fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
#    sizeref=0.1,colorscale=[[0, 'green'], [1,'green']])

#Spin Accumulation

#x1 = np.array([0,0])
#y1 = np.array([0,0.5])
#z1 = np.array([0,0])

#fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
#    marker=dict(size=0.1,color='black'),
#    line=dict(color='yellow', width=6),
#    name="$\sigma_{AC}$")
#fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
#sizeref=0.1,colorscale=[[0, 'yellow'], [1,'yellow']])


def update(t=0):
    with fig1.batch_update():
        f=0.9
        fig1.data[0].x=[0,f*X1s[t]]
        fig1.data[0].y=[0,f*Y1s[t]]
        fig1.data[0].z=[0,f*Z1s[t]]
        
        fig1.data[1].x=[X1s[t]]
        fig1.data[1].y=[Y1s[t]]
        fig1.data[1].z=[Z1s[t]]
        fig1.data[1].u=[2*X1s[t]]
        fig1.data[1].v=[2*Y1s[t]]
        fig1.data[1].w=[2*Z1s[t]]
        
        m=[X1s[t],Y1s[t],Z1s[t]]        
        DL=np.cross(m,np.cross(m,Y))
        DL=DL/3
        #DL=DL/(5*np.linalg.norm(DL))
        x1 = np.array([X1s[t],X1s[t]+DL[0]])
        y1 = np.array([Y1s[t],Y1s[t]+DL[1]])
        z1 = np.array([Z1s[t],Z1s[t]+DL[2]])
        
        

        


w=interact(update,t=widgets.IntSlider(min=0, max=len(t)-1, step=5, value=0,description="Time Step",readout_format=".1d",continuous_update=False))
#w=interactive(update,t=(0,number_of_points-1,1)) 
#w.children[0].description="$Time (t)$"
#display(w)
fig1



interactive(children=(IntSlider(value=0, continuous_update=False, description='Time Step', max=4999, readout_f…

FigureWidget({
    'data': [{'line': {'color': 'black', 'width': 4},
              'marker': {'color': 'black'…

In [11]:
t = smp.symbols('t', real=True)
Hx, Hy, Hz, a, g, FL, DL, Hex, Kx,Dz= smp.symbols('Hx Hy Hz a g H_{FL} H_{DL} H_{ex} K_{x} D_{z} ', real=True)
mx, my, mz = smp.symbols(r'm_{x} m_{y} m_{z}', cls=smp.Function)
mx = mx(t)
my = my(t)
mz = mz(t)
# Derivatives
mx_d = smp.diff(mx,t)
my_d = smp.diff(my,t)
mz_d = smp.diff(mz,t)

m=Matrix([mx,my,mz])

m_d=Matrix([mx_d,my_d,mz_d])
H=Matrix([Hx,Hy,Hz])

X=Matrix([1,0,0])
Y=Matrix([0,1,0])
Z=Matrix([0,0,1])

FE= -1*Kx*((X.dot(m))**2)+Dz*((Z.dot(m))**2) -H.dot(m)
Heff=smp.diff(FE,m)

llg=-m_d + g*(m.cross(Heff))+a*(m.cross(m_d))

sols= smp.solve(llg, (mx_d, my_d, mz_d),simplify=True, rational=False)
mx_df2 = smp.lambdify((t,m), sols[mx_d].subs([(g,g_value),(a,a_value),(Kx,Kx_value),(Dz,Dz_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))
my_df2 = smp.lambdify((t,m), sols[my_d].subs([(g,g_value),(a,a_value),(Kx,Kx_value),(Dz,Dz_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))
mz_df2 = smp.lambdify((t,m), sols[mz_d].subs([(g,g_value),(a,a_value),(Kx,Kx_value),(Dz,Dz_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))

def dSdt2(t,S):
    mx,my,mz  = S
    return [
        mx_df2(t,[mx,my,mz]),
        my_df2(t,[mx,my,mz]),
        mz_df2(t,[mx,my,mz]),
    ]



t_n=340
t_n=705
init_cond=(X1s[t_n],Y1s[t_n],Z1s[t_n])



t = np.linspace(0, 1000, 5000)

sol2 = odeint(dSdt2, y0=init_cond, t=t, tfirst=True)
sol2=sol2.T

In [12]:
X2s=sol2[0]
Y2s=sol2[1]
Z2s=sol2[2]

fig1 = go.FigureWidget()
#NEEL VECTOR1
x1 = np.array([0,X2s[0]])
y1 = np.array([0,Y2s[0]])
z1 = np.array([0,Z2s[0]])
f=0.95
fig1.add_scatter3d(x=x1*f, y=y1*f, z=z1*f,opacity=0.75,
    marker=dict(size=0.1,color='black'),
    line=dict(color='black', width=4 ),
    name="$m$",showlegend=False)
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
    sizeref=0.1,colorscale=[[0, 'black'], [1,'black']])


Y=[0,1,0]
m=[X2s[0],Y2s[0],Z2s[0]]
DL=np.cross(m,np.cross(m,Y))
DL=DL/(5*np.linalg.norm(DL))
x = np.array([X2s[0],X2s[0]+DL[0]])
y = np.array([Y2s[0],Y2s[0]+DL[1]])
z = np.array([Z2s[0],Z2s[0]+DL[2]])

#Damping-like Torque
#fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.9,
#    marker=dict(size=0.1,color='black'),
#    line=dict(color='brown', width=5 ),
#    name="$ \\tau_{DL}$",showlegend=True)

fig1.update_layout(
    height=700,
    width=900,

    autosize=True,
    scene=dict(camera=dict(up=dict(x=0,y=0, z=1 ), eye=dict(x=1,y=-1,z=1,)),
               xaxis_title='mx',
               yaxis_title='my',
               zaxis_title='mz',
               xaxis = dict(nticks=3, range=[-1,1]),
               yaxis = dict(nticks=3, range=[-1,1]),
               zaxis = dict(nticks=3, range=[-1,1]),
        aspectratio = dict( x=1, y=1, z=1),
        aspectmode = 'manual'))
#CIRCLE IN XY PLANE
C1=1
phi=np.linspace(0,2*np.pi,100)
x=C1*np.cos(phi)
y=C1*np.sin(phi)
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN XZ PLANE
phi=np.linspace(0,2*np.pi,100)
x=C1*np.cos(phi)
y=C1*np.zeros(len(x))
z=C1*np.sin(phi)
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

#CIRCLE IN YZ PLANE
phi=np.linspace(0,2*np.pi,100)
x=C1*np.zeros(len(x))
y=C1*np.cos(phi)
z=C1*np.sin(phi)
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=C1*np.zeros(len(x))
y=C1*np.zeros(len(x))
z=phi
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=C1*np.zeros(len(x))
y=phi
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=phi
y=C1*np.zeros(len(x))
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

# UNIT SPHERE
theta, phi = np.mgrid[0:2*np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.sin(phi)
y = np.sin(theta)*np.cos(phi)
z = np.cos(theta)

fig1.add_surface(x=x,y=y,z=z,opacity=0.07,colorscale=[[0, 'purple'], [1,'orange']],showlegend=False, showscale = False)

fig1.add_scatter3d(x=X2s, y=Y2s, z=Z2s,opacity=0.25,
    marker=dict(size=0.1,color='red'),
    line=dict(color='red', width=3),
    showlegend=False)

#External Field1
#norm=2*np.sqrt(float(Hx_value**2+Hy_value**2+Hz_value**2))
#x1 = np.array([0,Hx_value/norm])
#y1 = np.array([0,Hy_value/norm])
#z1 = np.array([0,Hz_value/norm])

#fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
#    marker=dict(size=0.1,color='black'),
#    line=dict(color='green', width=6 ),
#    name="$H_{applied}$")
#fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
#    sizeref=0.1,colorscale=[[0, 'green'], [1,'green']])

#Spin Accumulation

#x1 = np.array([0,0])
#y1 = np.array([0,0.5])
#z1 = np.array([0,0])

#fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
#    marker=dict(size=0.1,color='black'),
#    line=dict(color='yellow', width=6),
#    name="$\sigma_{AC}$")
#fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
#sizeref=0.1,colorscale=[[0, 'yellow'], [1,'yellow']])


def update(t=0):
    with fig1.batch_update():
        f=0.9
        fig1.data[0].x=[0,f*X2s[t]]
        fig1.data[0].y=[0,f*Y2s[t]]
        fig1.data[0].z=[0,f*Z2s[t]]
        
        fig1.data[1].x=[X2s[t]]
        fig1.data[1].y=[Y2s[t]]
        fig1.data[1].z=[Z2s[t]]
        fig1.data[1].u=[2*X2s[t]]
        fig1.data[1].v=[2*Y2s[t]]
        fig1.data[1].w=[2*Z2s[t]]
        
        m=[X2s[t],Y2s[t],Z2s[t]]        
        DL=np.cross(m,np.cross(m,Y))
        DL=DL/3
        #DL=DL/(5*np.linalg.norm(DL))
        x1 = np.array([X2s[t],X2s[t]+DL[0]])
        y1 = np.array([Y2s[t],Y2s[t]+DL[1]])
        z1 = np.array([Z2s[t],Z2s[t]+DL[2]])
        
        

        


w=interact(update,t=widgets.IntSlider(min=0, max=len(t)-1, step=5, value=0,description="Time Step",readout_format=".1d",continuous_update=False))
#w=interactive(update,t=(0,number_of_points-1,1)) 
#w.children[0].description="$Time (t)$"
#display(w)
fig1



interactive(children=(IntSlider(value=0, continuous_update=False, description='Time Step', max=4999, readout_f…

FigureWidget({
    'data': [{'line': {'color': 'black', 'width': 4},
              'marker': {'color': 'black'…

In [13]:
X3s=np.append(X1s[:t_n],X2s)
Y3s=np.append(Y1s[:t_n],Y2s)
Z3s=np.append(Z1s[:t_n],Z2s)

fig1 = go.FigureWidget()
#NEEL VECTOR1
x1 = np.array([0,X3s[0]])
y1 = np.array([0,Y3s[0]])
z1 = np.array([0,Z3s[0]])
f=0.95
fig1.add_scatter3d(x=x1*f, y=y1*f, z=z1*f,opacity=0.75,
    marker=dict(size=0.1,color='black'),
    line=dict(color='black', width=4 ),
    name="$m$",showlegend=False)
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
    sizeref=0.1,colorscale=[[0, 'black'], [1,'black']])


Y=[0,1,0]
m=[X3s[0],Y3s[0],Z3s[0]]
DL=np.cross(m,np.cross(m,Y))
DL=DL/(5*np.linalg.norm(DL))
x = np.array([X3s[0],X3s[0]+DL[0]])
y = np.array([Y3s[0],Y3s[0]+DL[1]])
z = np.array([Z3s[0],Z3s[0]+DL[2]])

#Damping-like Torque
#fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.9,
#    marker=dict(size=0.1,color='black'),
#    line=dict(color='brown', width=5 ),
#    name="$ \\tau_{DL}$",showlegend=True)

fig1.update_layout(
    height=700,
    width=900,

    autosize=True,
    scene=dict(camera=dict(up=dict(x=0,y=0, z=1 ), eye=dict(x=1,y=-1,z=1,)),
               xaxis_title='mx',
               yaxis_title='my',
               zaxis_title='mz',
               xaxis = dict(nticks=3, range=[-1,1]),
               yaxis = dict(nticks=3, range=[-1,1]),
               zaxis = dict(nticks=3, range=[-1,1]),
        aspectratio = dict( x=1, y=1, z=1),
        aspectmode = 'manual'))
#CIRCLE IN XY PLANE
C1=1
phi=np.linspace(0,2*np.pi,100)
x=C1*np.cos(phi)
y=C1*np.sin(phi)
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN XZ PLANE
phi=np.linspace(0,2*np.pi,100)
x=C1*np.cos(phi)
y=C1*np.zeros(len(x))
z=C1*np.sin(phi)
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

#CIRCLE IN YZ PLANE
phi=np.linspace(0,2*np.pi,100)
x=C1*np.zeros(len(x))
y=C1*np.cos(phi)
z=C1*np.sin(phi)
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=C1*np.zeros(len(x))
y=C1*np.zeros(len(x))
z=phi
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=C1*np.zeros(len(x))
y=phi
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)
#CIRCLE IN YZ PLANE
phi=np.linspace(-1,1,100)
x=phi
y=C1*np.zeros(len(x))
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

# UNIT SPHERE
theta, phi = np.mgrid[0:2*np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.sin(phi)
y = np.sin(theta)*np.cos(phi)
z = np.cos(theta)

fig1.add_surface(x=x,y=y,z=z,opacity=0.07,colorscale=[[0, 'purple'], [1,'orange']],showlegend=False, showscale = False)

fig1.add_scatter3d(x=X3s, y=Y3s, z=Z3s,opacity=0.25,
    marker=dict(size=0.1,color='red'),
    line=dict(color='red', width=3),
    showlegend=False)

#External Field1
#norm=2*np.sqrt(float(Hx_value**2+Hy_value**2+Hz_value**2))
#x1 = np.array([0,Hx_value/norm])
#y1 = np.array([0,Hy_value/norm])
#z1 = np.array([0,Hz_value/norm])

#fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
#    marker=dict(size=0.1,color='black'),
#    line=dict(color='green', width=6 ),
#    name="$H_{applied}$")
#fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
#    sizeref=0.1,colorscale=[[0, 'green'], [1,'green']])

#Spin Accumulation

#x1 = np.array([0,0])
#y1 = np.array([0,0.5])
#z1 = np.array([0,0])

#fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
#    marker=dict(size=0.1,color='black'),
#    line=dict(color='yellow', width=6),
#    name="$\sigma_{AC}$")
#fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
#sizeref=0.1,colorscale=[[0, 'yellow'], [1,'yellow']])


def update(t=0):
    with fig1.batch_update():
        f=0.9
        fig1.data[0].x=[0,f*X3s[t]]
        fig1.data[0].y=[0,f*Y3s[t]]
        fig1.data[0].z=[0,f*Z3s[t]]
        
        fig1.data[1].x=[X3s[t]]
        fig1.data[1].y=[Y3s[t]]
        fig1.data[1].z=[Z3s[t]]
        fig1.data[1].u=[2*X3s[t]]
        fig1.data[1].v=[2*Y3s[t]]
        fig1.data[1].w=[2*Z3s[t]]
        
        m=[X3s[t],Y3s[t],Z3s[t]]        
        DL=np.cross(m,np.cross(m,Y))
        DL=DL/3
        #DL=DL/(5*np.linalg.norm(DL))
        x1 = np.array([X3s[t],X3s[t]+DL[0]])
        y1 = np.array([Y3s[t],Y3s[t]+DL[1]])
        z1 = np.array([Z3s[t],Z3s[t]+DL[2]])
        
        

        


w=interact(update,t=widgets.IntSlider(min=0, max=len(t)-1, step=5, value=0,description="Time Step",readout_format=".1d",continuous_update=False))
#w=interactive(update,t=(0,number_of_points-1,1)) 
#w.children[0].description="$Time (t)$"
#display(w)
fig1



interactive(children=(IntSlider(value=0, continuous_update=False, description='Time Step', max=4999, readout_f…

FigureWidget({
    'data': [{'line': {'color': 'black', 'width': 4},
              'marker': {'color': 'black'…

In [14]:
pio.write_image(fig1,r"C:\Users\Egecan\Desktop\CaseII.png",height=1000,width=1200)

In [15]:
def customplot(X3s,Y3s,Z3s,t):

    fig1 = go.FigureWidget()
    #NEEL VECTOR1
    x1 = np.array([0,X3s[t]])
    y1 = np.array([0,Y3s[t]])
    z1 = np.array([0,Z3s[t]])
    f=0.95
    fig1.add_scatter3d(x=x1*f, y=y1*f, z=z1*f,opacity=0.75,
        marker=dict(size=0.1,color='black'),
        line=dict(color='black', width=4 ),
        name="$m$",showlegend=False)
    fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
        sizeref=0.1,colorscale=[[0, 'black'], [1,'black']])

    fig1.update_layout(
        height=700,
        width=700,

        autosize=True,
        scene=dict(camera=dict(up=dict(x=0,y=0, z=1 ), eye=dict(x=1.3,y=0.9,z=0.8,)),
                   xaxis_title='mx',
                   yaxis_title='my',
                   zaxis_title='mz',
                   xaxis = dict(nticks=3, range=[-1,1]),
                   yaxis = dict(nticks=3, range=[-1,1]),
                   zaxis = dict(nticks=3, range=[-1,1]),
            aspectratio = dict( x=1, y=1, z=1),
            aspectmode = 'manual'))
    #CIRCLE IN XY PLANE2
    C1=1
    phi=np.linspace(0,2*np.pi,100)
    x=C1*np.cos(phi)
    y=C1*np.sin(phi)
    z=C1*np.zeros(len(x))
    fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
        marker=dict(size=0.1,color='grey'),
        line=dict(color='grey', width=2 ),
        showlegend=False)
    #CIRCLE IN XZ PLANE
    phi=np.linspace(0,2*np.pi,100)
    x=C1*np.cos(phi)
    y=C1*np.zeros(len(x))
    z=C1*np.sin(phi)
    fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
        marker=dict(size=0.1,color='grey'),
        line=dict(color='grey', width=2 ),
        showlegend=False)

    #CIRCLE IN YZ PLANE
    phi=np.linspace(0,2*np.pi,100)
    x=C1*np.zeros(len(x))
    y=C1*np.cos(phi)
    z=C1*np.sin(phi)
    fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
        marker=dict(size=0.1,color='grey'),
        line=dict(color='grey', width=2 ),
        showlegend=False)

    #CIRCLE IN YZ PLANE
    phi=np.linspace(-1,1,100)
    x=C1*np.zeros(len(x))
    y=C1*np.zeros(len(x))
    z=phi
    fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
        marker=dict(size=0.1,color='grey'),
        line=dict(color='grey', width=2 ),
        showlegend=False)
    #CIRCLE IN YZ PLANE
    phi=np.linspace(-1,1,100)
    x=C1*np.zeros(len(x))
    y=phi
    z=C1*np.zeros(len(x))
    fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
        marker=dict(size=0.1,color='grey'),
        line=dict(color='grey', width=2 ),
        showlegend=False)
    #CIRCLE IN YZ PLANE
    phi=np.linspace(-1,1,100)
    x=phi
    y=C1*np.zeros(len(x))
    z=C1*np.zeros(len(x))
    fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.1,
        marker=dict(size=0.1,color='grey'),
        line=dict(color='grey', width=2 ),
        showlegend=False)

    # UNIT SPHERE
    theta, phi = np.mgrid[0:2*np.pi:100j, 0:2*np.pi:100j]
    x = np.sin(theta)*np.sin(phi)
    y = np.sin(theta)*np.cos(phi)
    z = np.cos(theta)

    fig1.add_surface(x=x,y=y,z=z,opacity=0.07,colorscale=[[0, 'purple'], [1,'orange']],showlegend=False, showscale = False)

    fig1.add_scatter3d(x=X3s, y=Y3s, z=Z3s,opacity=0.25,
        marker=dict(size=0.1,color='red'),
        line=dict(color='red', width=3),
        showlegend=False)

    if t<10:
        name="000"+str(t)
    elif t<100:
        name="00"+str(t)
    elif t<1000:
        name="0"+str(t)
    elif t<10000:
        name=str(t)
    pio.write_image(fig1,r"C:\Users\Egecan\Desktop\CaseB\\"+name+".png",height=1000,width=1200)

    
    
    
    

In [16]:
#customrange=np.arange(1300,3000,20)
#for i in customrange:
#    customplot(X3s,Y3s,Z3s,i)